# Project: Investigating Soccer Player Interceptions

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> This project will use multiple regression analyses to see if soccer player interceptions have any type of relationship with other attributes a player has during a game.  

### Dataset

> The data for this project was obtained from a Udacity sqlite database (https://docs.google.com/document/d/e/2PACX-1vTlVmknRRnfy_4eTrjw5hYGaiQim5ctr9naaRd4V9du2B5bxpd8FEH3KtDgp8qVekw7Cj1GLk1IXdZi/pub?embedded=True).

> The Udacity database included a table called player_attributes where the interceptions value was measured.  This table was used as the main dataset for the study.  Another table called player was also combine with the player_attributes dataset to add the additional value of birthday.

> Below is the list of attributes that were compared to interceptions:

> 'date', 'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle’, ‘gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'birthday'


In [187]:
# Use this cell to set up import statements for all of the packages that you
#   plan to use.

# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html

import sqlite3
import pandas as pd
import numpy as np
% matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

> The first step in this project is to gather and review the data. 

> The data was provided to me from Udacity in a sqlite database. I used the database to generate a python DataFrame.

In [188]:
# Connect to the soccer database
conn = sqlite3.connect("database.sqlite")

# Read database and save as panda
df = pd.read_sql_query("""SELECT player_attributes.*,
                                 player.birthday
                          FROM player_attributes
                          JOIN player ON player_attributes.player_fifa_api_id = player.player_fifa_api_id""", conn)

print(df.columns)
print(df.head())

# We've got everything we need from the database so close the database connection
conn.close()

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes', 'birthday'],
      dtype='object')
   id  player_fifa_api_id  player_api_id                 date  overall_rating  \
0   1              218353         505942  2016-02-18 00:00:00            67.0   
1   2              218353         505942  2015-11-19 00:00:00            67.0   
2   3              218353         

### Review the data
> Once the data was collected, I reviewed the dataset.

> The data had some null values.  These values accounted for less than 2% of the total dataset.  Since the missing values were not significant, I chose to remove the values. 

> The date column and birthday column were transformed from string values to date values.  Since birthday is not a changing number, birthday was used to instead calculate player age at the time of the game date.

> No duplicates rows were found in the dataset.

In [189]:
# Check for missing values
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183929 entries, 0 to 183928
Data columns (total 43 columns):
id                     183929 non-null int64
player_fifa_api_id     183929 non-null int64
player_api_id          183929 non-null int64
date                   183929 non-null object
overall_rating         183142 non-null float64
potential              183142 non-null float64
preferred_foot         183142 non-null object
attacking_work_rate    180748 non-null object
defensive_work_rate    183142 non-null object
crossing               183142 non-null float64
finishing              183142 non-null float64
heading_accuracy       183142 non-null float64
short_passing          183142 non-null float64
volleys                181265 non-null float64
dribbling              183142 non-null float64
curve                  181265 non-null float64
free_kick_accuracy     183142 non-null float64
long_passing           183142 non-null float64
ball_control           183142 non-null float64
accele

In [191]:
# There are 183,929 columns in this data set.  It looks like almost all of our columns have some null values.

# For this project, I am going to drop the null values. 
# Even with the dropped values, our dataset will have 180,354 values left for calcuation which is a large sample size. 
# Also, the missing values account for less than 2% of the original dataset.

df.dropna(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180354 entries, 0 to 183928
Data columns (total 43 columns):
id                     180354 non-null int64
player_fifa_api_id     180354 non-null int64
player_api_id          180354 non-null int64
date                   180354 non-null object
overall_rating         180354 non-null float64
potential              180354 non-null float64
preferred_foot         180354 non-null object
attacking_work_rate    180354 non-null object
defensive_work_rate    180354 non-null object
crossing               180354 non-null float64
finishing              180354 non-null float64
heading_accuracy       180354 non-null float64
short_passing          180354 non-null float64
volleys                180354 non-null float64
dribbling              180354 non-null float64
curve                  180354 non-null float64
free_kick_accuracy     180354 non-null float64
long_passing           180354 non-null float64
ball_control           180354 non-null float64
accele

In [192]:
# Incorrect Datatypes

# I'm going to change date fields from strings to dates for better data manipulation. 
df['date'] = pd.to_datetime(df['date'])
df['birthday'] = pd.to_datetime(df['birthday'])

# Birthday isn't super useful.  What I really want is the player's age during the game
# Stack Overflow helped with the calcuation
df['age_during_game'] =  (df['date'] - df['birthday']).apply(lambda age_in_days: int(float(x.days)/365))

print(df.head(5))

   id  player_fifa_api_id  player_api_id       date  overall_rating  \
0   1              218353         505942 2016-02-18            67.0   
1   2              218353         505942 2015-11-19            67.0   
2   3              218353         505942 2015-09-21            62.0   
3   4              218353         505942 2015-03-20            61.0   
4   5              218353         505942 2007-02-22            61.0   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0       71.0          right              medium              medium      49.0   
1       71.0          right              medium              medium      49.0   
2       66.0          right              medium              medium      49.0   
3       65.0          right              medium              medium      48.0   
4       65.0          right              medium              medium      48.0   

        ...         marking  standing_tackle  sliding_tackle  gk_diving  \
0       ...

In [185]:
# Check for duplicate values
df.duplicated().sum()
# It looks like we do not have any duplicate rows in this dataset 

0

In [186]:
# let's also check player_fifa_api_id and date.  Ideally, one player should only have one set of scores per game
df.groupby(['player_fifa_api_id', 'date']).count().max
# It looks like each player appears only once per date so we don't have any duplicate player data in the set.

<bound method DataFrame.max of                                id  player_api_id  overall_rating  potential  \
player_fifa_api_id date                                                       
2                  2007-02-22   1              1               1          1   
                   2007-08-30   1              1               1          1   
                   2008-08-30   1              1               1          1   
                   2009-02-22   1              1               1          1   
                   2009-08-30   1              1               1          1   
                   2010-02-22   1              1               1          1   
                   2010-08-30   1              1               1          1   
                   2011-02-22   1              1               1          1   
                   2011-08-30   1              1               1          1   
                   2012-02-22   1              1               1          1   
                   20

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [4]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])

0